In [1]:
import pandas as pd 
import scipy as sp
from scipy import stats
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.kernel_ridge import KernelRidge
import math
from scipy.signal import argrelextrema


RET_DAYS = 1 # How many days of return we are looking at

In [2]:
def findLocalExtrema(m):

    maxInd = argrelextrema(m, np.greater)[0]
    minInd = argrelextrema(m, np.less)[0]
    exmInd = np.sort(np.concatenate([maxInd,minInd]))
    indicator = np.zeros(len(exmInd)) # indicate whether the extremum is a max or a min
    
    for i in range(len(exmInd)):
        if exmInd[i] in maxInd:
            indicator[i] = 1
        elif exmInd[i] in minInd:
            indicator[i] = -1
           
    return exmInd,indicator


In [3]:
def checkHS(m,exmInd,indicator):

    flag = False
    pid = None
    nexm = len(exmInd)
    exmv = m[exmInd]
    for i in range(nexm-4):
        upper = (exmv[i] + exmv[i+4])/2
        lower = (exmv[i+1] + exmv[i+3])/2
        if (indicator[i] > 0) and (exmv[i+2]> exmv[i]) and (exmv[i+2]>exmv[i+4]) \
        and (abs((exmv[i] - upper)/upper) <= 0.015) and (abs((exmv[i+4] - upper)/upper) <= 0.015) \
        and (abs((exmv[i+1] - lower)/lower) <= 0.015) and (abs((exmv[i+3] - lower)/lower) <= 0.015):
            flag = True
            pid = exmInd[np.arange(i,i+5)] # this is the index in the exmInd 
            break

    return flag, pid
        
def checkIHS(m,exmInd,indicator):
   
    flag = False
    pid = None
    nexm = len(exmInd)
    exmv = m[exmInd]
    for i in range(nexm-4):
        upper = (exmv[i] + exmv[i+4])/2
        lower = (exmv[i+1] + exmv[i+3])/2
        if (indicator[i] < 0) and (exmv[i+2] < exmv[i]) and (exmv[i+2]<exmv[i+4]) \
        and (abs((exmv[i] - upper)/upper) <= 0.015) and (abs((exmv[i+4] - upper)/upper) <= 0.015) \
        and (abs((exmv[i+1] - lower)/lower) <= 0.015) and (abs((exmv[i+3] - lower)/lower) <= 0.015):
            flag = True
            pid = exmInd[np.arange(i,i+5)]
            break

    return flag, pid

In [4]:
def checkBTOP(m,exmInd,indicator):
    
    flag = False
    pid = None
    nexm = len(exmInd)
    exmv = m[exmInd]
    for i in range(nexm-4):
        if (indicator[i] > 0) and (exmv[i] < exmv[i+2]) and (exmv[i+2]<exmv[i+4]) and (exmv[i+1] > exmv[i+3]):
            flag = True
            pid = exmInd[np.arange(i,i+5)]
            break
       
    return flag, pid


def checkBBOT(m,exmInd,indicator):
 
    flag = False
    pid = None
    nexm = len(exmInd)
    exmv = m[exmInd]
    for i in range(nexm-4):
        if (indicator[i] < 0) and (exmv[i] > exmv[i+2]) and (exmv[i+2]>exmv[i+4]) and (exmv[i+1] < exmv[i+3]):
            flag = True
            pid = exmInd[np.arange(i,i+5)] # this is the index in the exmInd 
            break
       
    return flag, pid

In [5]:
def checkTTOP(m,exmInd,indicator):

    flag = False
    pid = None
    nexm = len(exmInd)
    exmv = m[exmInd]
    for i in range(nexm-4):
        if (indicator[i] > 0) and (exmv[i] > exmv[i+2]) and (exmv[i+2]>exmv[i+4]) and (exmv[i+1] < exmv[i+3]):
            flag = True
            pid = exmInd[np.arange(i,i+5)] # this is the index in the exmInd 
            break
    return flag, pid


def checkTBOT(m,exmInd,indicator):

    flag = False
    pid = None
    nexm = len(exmInd)
    exmv = m[exmInd]
    for i in range(nexm-4):
        if (indicator[i] < 0) and (exmv[i] < exmv[i+2]) and (exmv[i+2]<exmv[i+4]) and (exmv[i+1] > exmv[i+3]):
            flag = True
            pid = exmInd[np.arange(i,i+5)] # this is the index in the exmInd 
            break
       
    return flag, pid

In [6]:
def checkRTOP(m,exmInd,indicator):

    flag = False
    pid = None
    nexm = len(exmInd)
    exmv = m[exmInd]
    top = exmInd[indicator>0]
    bot = exmInd[indicator<0]
    avgtop = np.mean(top)
    avgbot = np.mean(bot)
    for i in range(nexm-4):
        if (indicator[i] > 0) and (all(top<avgtop*(1+0.075))) and (all(top>avgtop*(1-0.075)))\
        and (all(bot<avgbot*(1+0.075))) and (all(bot>avgbot*(1-0.075)))\
        and (min(top)>max(bot)):
            flag = True
            pid = exmInd[np.arange(i,i+5)] # this is the index in the exmInd 
            break
       
    return flag, pid


def checkRBOT(m,exmInd,indicator):

    flag = False
    pid = None
    nexm = len(exmInd)
    exmv = m[exmInd]
    top = exmInd[indicator>0]
    bot = exmInd[indicator<0]
    avgtop = np.mean(top)
    avgbot = np.mean(bot)
    for i in range(nexm-4):
        if (indicator[i] < 0) and (all(top<avgtop*(1+0.075))) and (all(top>avgtop*(1-0.075)))\
        and (all(bot<avgbot*(1+0.075))) and (all(bot>avgbot*(1-0.075)))\
        and (min(top)>max(bot)):
            flag = True
            pid = exmInd[np.arange(i,i+5)] # this is the index in the exmInd 
            break
       
    return flag, pid

In [7]:
def checkDTOP(m,exmInd,indicator):

    flag = False
    pid = None
    
    if len(exmInd)<=1:
        pass
    else:    
        exmv = m[exmInd]
        ea = max(exmv[1:])
        avge = (ea + exmv[0])/2
        ta = np.argmax(exmv[1:])+1
        ia = exmInd[ta]
        if (indicator[0]>0) and (ia>exmInd[0]+22) and (abs((exmv[0]-avge)/avge)<=0.015)and (abs((ea-avge)/avge)<=0.015):
            flag = True
            pid =  np.array([exmInd[0],ia]) # this is the index in the exmInd
    
    return flag, pid


def checkDBOT(m,exmInd,indicator):

    flag = False
    pid = None
    
    if len(exmInd)<=1:
        pass
    else:
        exmv = m[exmInd]
        ea = min(exmv[1:])
        avge = (ea + exmv[0])/2
        ta = np.argmin(exmv[1:])+1
        ia = exmInd[ta]
        if (indicator[0]<0) and(ia>exmInd[0]+22) and (abs((exmv[0]-avge)/avge)<=0.015)and (abs((ea-avge)/avge)<=0.015):
            flag = True
            pid =  np.array([exmInd[0],ia]) # this is the index in the exmInd   
    
    return flag, pid

In [8]:
'''
def GuassianKernal(x,h):
    return stats.norm.pdf(x,loc=0,scale=h)
'''
def GuassianKernal(x,h):
    #return stats.norm.pdf(x,loc=0,scale=h)
    return 1.0/h/np.sqrt(2*np.pi) * np.exp(-x*x/2.0/h/h)


def KernelSmooth(x, signal, h, kernel):
    m = np.zeros(len(x))
    m_hat = np.zeros(len(x))
    for t in range(len(x)):
        this_x = x[t]
        k = kernel(this_x-x,h)
        
        g = np.mean(k)
        w = k/g
        m[t] = np.mean(w*signal)
        m_hat[t] = (np.sum(w*signal)-w[t]*signal[t])/len(x)
        cv = np.mean((signal-m_hat)**2)
    return m, m_hat, cv 

from scipy.optimize import minimize
def chooseH(x, signal, kernel):
    def objFunc(h):
        m,m_hat, cv = KernelSmooth(x, signal, h, kernel)
        return cv
    res = minimize(objFunc, 1, 
                   method='Nelder-Mead', tol=1e-6, 
                  bounds = (0, None)
                  )
    m_, m_hat_, cv_ = KernelSmooth(x, signal, res.x[0], kernel)
    return res.x[0], m_, m_hat_, cv_ 

In [9]:
def getSampleData(period_num, cap_num,usePrice = True):
    fileName = r'E:\Final_Project\Data\\samples\\period%s_cap%s.xlsx' % (str(period_num), str(cap_num))
    sheetname = 'price' if usePrice else 'return'
    data = pd.read_excel(fileName, sheetname = sheetname, index_col = 0)
    return data

In [10]:
sample_data = getSampleData(1,1,True)
sample_data.tail()

BadZipfile: File is not a zip file

In [ ]:
patternLib = ['HS', 'IHS','BTOP', 'BBOT', 'TTOP', 'TBOT', 'RTOP', 'RBOT', 'DTOP', 'DBOT']
luckyDay = {x:[] for x in patternLib}

In [ ]:
def findPattern_OneTS(ts, h = 3):
    l = 35
    d = 3
    window = l+d
    nobs = len(ts)
    #nobs = 1000
    x = np.arange(len(ts))
    i = 0
    luckyDay = {x:[] for x in patternLib}
    while i+window<=nobs:
        sample_signal = ts.values[i:i+window]
        sample_x = np.arange(i,i+window)
        sample_m,_,_ = KernelSmooth(sample_x, sample_signal, h, GuassianKernal)
        exmInd,indicator = findLocalExtrema(sample_m[0:l])
        findAny = False
        firstExtremaIdx = 999999
        firstPattern = None
        earliestLastextrema = None
        # see if we can find any patterns
        for thisPattern in patternLib:
            call_function = 'check'+ thisPattern + '(sample_m[0:l],exmInd,indicator)'
            flag,pid = eval(call_function)
            findAny  = findAny or flag
            # if find, we need to only use the first pattern we find
            if flag:
                if pid[0] < firstExtremaIdx:
                    firstExtremaIdx = pid[0]
                    earliestLastextrema = pid[-1]
                    firstPattern = thisPattern
        # Record the return on the day after the period as a conditional return for the first pattern found
        if findAny:
            try:
                luckyDay[firstPattern] += [ts.index[i+window+1]]
                #luckyDay[firstPattern] = luckyDay[firstPattern] + [ts.index[i+earliestLastextrema+1]] 
            except:
                pass
            i += (firstExtremaIdx+1) 
        else:
            i += 1
    return luckyDay

In [ ]:
#findPattern_OneTS(sample_data.ix[:, 0], h = 1.5)

In [ ]:
def multiDayReturn(ts, days = 3):
    if days == 1:
        return ts
    else:
        tmp = (ts+1.).cumprod()
        return tmp.pct_change(days).shift(-days)

In [ ]:
def getCondRet_OneSample(period_num = 1, cap_num =1, h = 3, days = 1):
    sample_price = getSampleData(period_num,cap_num,True)
    sample_return = getSampleData(period_num,cap_num,False)
    multiday_ret = multiDayReturn(sample_return, days = days)
    condReturn = {x:[] for x in patternLib}
    for thisStock in sample_price:
        ret_mean = multiday_ret[thisStock].mean()
        ret_sig = multiday_ret[thisStock].std()
        thisDates = findPattern_OneTS(sample_price[thisStock], h = h)
        for thisPattern in patternLib:
            condReturn[thisPattern] += [(multiday_ret.loc[date1, thisStock]-ret_mean  )/ret_sig  #
                                        for date1 in thisDates[thisPattern]]
    return condReturn

def drawHistogram(period1_cap1):
    for x in patternLib:
        tmp = period1_cap1[x]
        tmp = np.array([k for k in period1_cap1[x] if not np.isnan(k)])
        print x, tmp.mean()
        plt.hist(tmp, bins = 20)
        plt.show()

In [ ]:
period1_cap1 = getCondRet_OneSample(1,1, 1.5, RET_DAYS)
drawHistogram(period1_cap1)

In [ ]:
period1_cap2 = getCondRet_OneSample(1,2, 1.5, RET_DAYS)
drawHistogram(period1_cap2)

In [ ]:
period1_cap3 = getCondRet_OneSample(1,3, 1.5, RET_DAYS)
drawHistogram(period1_cap3)

In [ ]:
period1_cap4 = getCondRet_OneSample(1,4, 1.5, RET_DAYS)
drawHistogram(period1_cap4)

In [ ]:
period1_cap5 = getCondRet_OneSample(1,5, 1.5, RET_DAYS)
drawHistogram(period1_cap5)

In [ ]:
sample_price

In [ ]:
def uncondReturn(period_num = 1, cap_num =1, days = 1):
    sample_price = getSampleData(period_num,cap_num,True)
    sample_return = getSampleData(period_num,cap_num,False)
    multiday_ret = multiDayReturn(sample_return, days = days)
    norm_ret = (multiday_ret - multiday_ret.mean())/ multiday_ret.std() #  
    norm_ret = norm_ret.as_matrix().flatten()
    norm_ret = norm_ret[~np.isnan(norm_ret)]
    return norm_ret
    

In [ ]:
uncondRet = uncondReturn(period_num = 1, cap_num =1 , days = RET_DAYS)

In [ ]:
print uncondRet.mean()

In [ ]:
plt.hist(uncondRet)

In [ ]:
np.array(period1_cap1['DBOT']).mean()

In [ ]:
sample_data = pd.read_csv(r"TYC.csv", index_col = 0)['PRC']
sample_data.index = range(len(sample_data))
x = np.arange(len(sample_data))
signal = sample_data.values
#h_min, _, _, _  = chooseH(x, signal, GuassianKernal)
h_min = 10
#m,m_hat,cv = KernelSmooth(x, signal, h_min * 0.15, GuassianKernal)
#plt.plot(m)
#plt.show()

In [ ]:
l = 35
d = 3
window = l+d
#nobs = len(x)
nobs = 1000

i = 0
while i+window<=nobs:

    sample_signal = signal[i:i+window]
    sample_x = x[i:i+window]
    sample_m,_,_ = KernelSmooth(sample_x, sample_signal, h_min*0.15, GuassianKernal)
    
    exmInd,indicator = findLocalExtrema(sample_m[0:l])
    
    flag,pid = checkDBOT(sample_m[0:l],exmInd,indicator)
    if flag:
        print 'pattern detected'+str(i)+str(pid)
        print sample_m[pid]
        plt.plot(sample_m)
        plt.plot(pid,sample_m[pid],'ro')
        plt.show()
        i = pid[0]+i
    else:
        i = i+1
            
            
    

In [ ]:
### old code

l = 35
d = 3
window = l+d
#nobs = len(x)
nobs = 1000
for i in range(nobs-window):

    sample_signal = signal[i:i+window]
    sample_x = x[i:i+window]
    sample_m,_,_ = KernelSmooth(sample_x, sample_signal, h_min*0.15, GuassianKernal)
    flag,pid = checkHS(sample_m[0:l])
    if flag:
        print 'pattern detected'+str(i)+str(pid)
        for pid_ in pid:
            print sample_m[pid_]
            plt.plot(sample_m)
            plt.plot(pid_,sample_m[pid_],'ro')
            plt.show()